# Lab 6: Logistic Regression, AUC and Random Forests

In lecture on ML, we argued that given enough data, Logisitic Regression should always do at least as well as Naive Bayes on binary-feature data. Lets try that out on the text dataset "data.tar.gz". You can download it from <a href="https://github.com/data-boss/Data-Science.git">here</a> and do
<pre>tar xvf data.tar.gz
</pre>

which produces an "data" directory with the data files in it. We'll also need the MNIST data from the last lab. Unpack if needed and do:
<pre>tar xvzf MNIST.tar.gz
</pre>

Start IPython, and load the text data:

In [2]:
import numpy as np
import scipy.sparse as sp
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline

iwords = np.loadtxt("data/words.imat.txt")          # training data matrix in nnz x 3 form - rows are (doc, word, count) triples
traincats = np.loadtxt("data/cats.imat.txt")        # training labels in an ntrain x ncats matrix
tiwords = np.loadtxt("data/testwords.imat.txt")     # test data matrix in nnz x 3 form
testcats = np.loadtxt("data/testcats.imat.txt")     # test labels

The data come as dense matrices with (row, col, val) triples in their rows. But they represent sparse matrices so we do the conversion next. Note that the matrix constructor uses wordcount>0 tests instead of the actual word counts which has the effect of making the word features binary. 

In [4]:
train = sp.csr_matrix((iwords[:,2] > 0, (iwords[:,1].astype(int), iwords[:,0].astype(int))))
ntrain = train.shape[0]
nfeats = train.shape[1]

test = sp.csr_matrix((tiwords[:,2] > 0, (tiwords[:,1], tiwords[:,0])),shape=(4000,nfeats))  # need to match the number of cols (words)
ntdocs = test.shape[0]

Once again we will concentrate on one label category, category 6.

In [5]:
traincat6 = traincats[:,6]
testcat6 = testcats[:,6]

Now we'll import a Logistic Regression Classifier

In [6]:
from sklearn.linear_model import LogisticRegression
lrclassifier = LogisticRegression()

and train it on the data:

In [ ]:
lrclassifier.fit(train,traincat6)

In [ ]:
preds = lrclassifier.predict(test)

> TODO: compute the accuracy of the predictions below: 
<br>
> What is the accuracy of your classifier model? Answer Q-1 in the response form

In [ ]:
# add code to score here

> TODO: How does this compare with Naive Bayes? In case you dont have the results handy, lets do it here:
<br>
Answer Q-2 in the response form

In [ ]:
from sklearn.naive_bayes import BernoulliNB
# add code to train, predict, score here

## Beyond Accuracy: ROC and AUC

Label prediction accuracy is a useful but sometimes misleading measure. e.g. for data with 10% positives, a predictor that always says "no" will be 90% accurate. Its also very often useful to control the ratio of positive/negative labels to minimize a loss function. e.g. false positives are generally more acceptable in computational marketing (it means you show an ad to someone who might not be interested) than false negatives (you failed to show an ad to someone who might be interest, and might generate some revenue for you). 

Logistic Regression computes the probability of a label and that output is useful for both richer evaluation methods, and for making more careful tradeoffs between positives and negatives. 

The ROC (Receiver-Operator Characteristic) curve is a very useful tool for interpreting classifier performance. See the background material here:
https://en.wikipedia.org/wiki/Receiver_operating_characteristic
It shows the classifiers TPR (True-Positive Rate) vs FPR (False-Positive Rate) at various thresholds. The threshold isnt shown on the plot but can be inferred later. TPR and FPR are defined as:

* TPR = TP / (TP + FN)   # based only on actual positive instances
* FPR = FP / (FP + TN)   # based only on actual negative instances

where TP = true positive, FN = false negative (actually a positive which is mislabelled) etc. 
Neither quantity involves a mix of positives and negatives. So ROC curves are insensitive to the actual ratio of positives to negatives. 

To use ROC, we first use a modified version of the ".predict" method which returns label probabilities:

In [ ]:
preds = lrclassifier.predict_proba(test);
preds.shape

From which you can see that there are 2 columns, i.e. one probability of false and one for true. Verify that the sum of every row is 1:

In [ ]:
np.sum(preds,axis=1)

We want the probabilities of cat6 membership = true, which is column 1:

In [ ]:
preds6 = preds[:,1]

and we'll do a ROC plot for it. ROC plots represent the performance of a classifier over a range of possible threshold values, showing the true positive rate and false positives rates at those thresholds. In Python, do

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
rc = roc_curve(testcat6, preds6)

This function returns the X and Y coordinates of the ROC plot. To see it, do:

In [ ]:
plt.plot(rc[0],rc[1])

> TODO: What is the True positive rate at an FPR of 0.2 ? 
<br>
> Q-3 

Finally, the ROC curve is sometimes too much information, especially if you want to compare performance of many classifiers or datasets. The overall performance is well-characterized by the AUC or Area Under the Curve. Which is exactly what the name suggests, the area under the blue curve. Since a ROC plot lies in a 1 x 1 square, the area is always <= 1.0. A random predictor puts positives and negatives on a diagonal line with slope = 1, and so a random predictor has AUC = 0.5

Lets check the AUC for our prediction:

In [ ]:
roc_auc_score(testcat6, preds6)

## Critical Thinking: Interpreting AUC scores

The AUC score varies between 0.5 (random prediction) and 1.0. A common misconception is that a "perfect" predictor, i.e. a predictor that knows the exact probability of a label, will give a score of 1.0. That's incorrect. There are two sources of noise in the generation of a ROC plot:
* The difference between the true and predicted probability of a label
* The variance introduced by Bernoulli sampling to generate the label

the latter is always present and depends on the distribution of label probabilities, the former depends on how good the model is. 

To see this, imagine a binary label distribution where each data label has a true probability of 0.5. A perfect predictor knows these probabilities but since they all the same, the sorted labels for the ROC plot would still be a random distribution of true and false. The ROC plot would have an AUC of 0.5. AUC scores very close to 1 are possible, but require that the true label distribution include a large fraction of probabilities close to either 1 or 0. That's because the variance of a Bernoulli variable is p(1-p), which is small if p is near 0 or 1. 

Lets estimate the ROC AUC for a perfect predictor on a similar distribution to our dataset. We cant know this distribution, but we can use the model's prediction  as an approximation to it. 

First we'll generate some uniform random numbers in [0,1], one for each test point. 

In [ ]:
a = npr.random(testcat6.shape)

Next we'll generate Bernoulli random numbers using the predictions as the underlying probability. We use the random numbers we just generated to do that. i.e. to generate a random Bernoulli variable with probability p, you generate a uniform random variable in [0,1] and test if (u < p). The probability that this test succeeds is exactly p. 

In [ ]:
x = (a < preds6)

In [ ]:
roc_auc_score(x, preds6)

Compare this number with the AUC you computed earlier. To be clear again what this number is, it is the score of a *perfect* label predictor with the label probability distribution that our classifier has. It is an estimate of how well our classifier could do on this dataset. 

This secondary AUC calculation is a useful normalizing test when interpreting AUC scores. A common mistake is to assume that a model with AUC 0.85 on dataset A is better (i.e. would score higher on a common dataset) than a model with a score of 0.70 on dataset B. This is not true. It depends strongly on the dataset. The model with score 0.70 may be generating perfect or near-perfect predictions. 

In [ ]:
plt.hist(np.log10(preds6),50)

Which is an enormous range of values. Most probabilities are very close to zero or one, which is why this dataset has such a high ROC AUC score.

Suppose instead we had a dataset with a less wide distribution. We can use a lognormal distribution to simulate this for us:

In [ ]:
cprob = np.minimum(npr.lognormal(-4,1,10000),1.0)

This is a pretty good model, e.g. for the range of user's probabilities of clicking on an ad. Lets look at a histogram of the log10 of the values (a direct histogram will be too squashed near 1). 

In [ ]:
plt.hist(np.log10(cprob),20)

So that's our distribution of virtual users. Notice that the values (which represent click probabilities) range over several orders of magnitude since we plotted their log10. Next we simulate users' click behavior. Once again we generate a uniform random variable u for each user, and output 1 if u < the user's click probability given by cprob. 

Finally we compute the AUC on that data, which is the score of a perfect predictor on this data.

In [ ]:
a = npr.random(cprob.shape)
x = (a < cprob)
roc_auc_score(x, cprob)

So that's the AUC score for a perfect predictor on this (artificial) dataset. This is lower than the *real* predictions on the RCV1 text dataset. So be careful when interpreting AUC scores. There is no absolute scale for them, and they depend a lot on the dataset.

Another important point is that the AUC value for mid-range scores can have quite a lot of variance. Try re-evaluating the last cell to see what happens. 

> TODO: What changes do you think you should make to the distribution cprob to increase the ROC AUC score?
<br>
> Q-4 

## Random Forests

Random Forests are an extremely accurate classifier for datasets of moderate size. Lets try them out here. We'll load the MNIST data now, but first its probably a good idea to restart your kernel to reduce memory use. Click on the "Kernel" menu above and then "Restart". 

In [ ]:
import numpy as np
import scipy.sparse as sp
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train0=np.loadtxt("MNIST/train.fmat.txt")
test0=np.loadtxt("MNIST/test.fmat.txt")
train = np.transpose(train0[:,0:4000])
test = np.transpose(test0[:,0:2000])
traincats = np.loadtxt("MNIST/ictrain.imat.txt")
testcats = np.loadtxt("MNIST/ictest.imat.txt")

Because we're going to tuning the parameters of RFs on some test data, we need to split our test set into a validation set and a final test set to avoid overfitting:

In [ ]:
validation = test[0:1000,:]
finaltest = test[1000:2000,:]
validationcats = testcats[0:1000]
finaltestcats = testcats[1000:2000]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclassifier = RandomForestClassifier(criterion='gini',max_features=30,n_estimators=20,n_jobs=4,bootstrap=True)
rfclassifier

In [ ]:
rfclassifier.fit(train,traincats)
preds = rfclassifier.predict(validation)
np.mean(preds == validationcats)

Read the scikit-learn documentation for Random Forests to make sure you understand the meaning of all the parameters in the call to the RandForestClassifier constructor. Which ones do you think will improve accuracy the most? **NOTE** you dont need to tune n_jobs. Its the number of threads that the classifier code runs and it only affects running time. It should be set to the number of cores that your VM has. 

Try tuning the classifier with the validation set above to get better than 90% accuracy on the validation set. Dont touch the final test set until you're done tuning. 

> TODO: Make a table with at least two values you tried each for criterion, max_features, n_estimators, and bootstrap. What trends to you notice for each one? 
<br>
> Q-5: Make a table in MSWORD and insert the file in response form.

> TODO: report your validation and final test accuracy.Include all the parameters you used, e.g. include the line where you invoked the RandomForestClassifier constructor. 
<br>
> Q-6: What is validation and final accuracy of random forest classifier for your selected parameters?

In [ ]:
preds = rfclassifier.predict(finaltest)
np.mean(preds == finaltestcats)

> TODO: Reflect on and Explain any differences between your validation and final test accuracy scores. 

> Q-7: What does the difference between validation and final test accuracy signify? Answer in terms of overfitting/underfitting and label distribution.

## Submit Your Responses

Use the lab response form <a href="https://forms.office.com/Pages/ResponsePage.aspx?id=bAnfdXKL5Eibkcv3nYfuOlHhusHecfNNjSokZ153wqVUM1NKTDdFTEtXOTlITzVBT1lVT1lSV0gyTi4u">here</a>. 